In [1]:
import torch
import torchvision

In [2]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [3]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [5]:
from PIL import Image
import numpy as np
from io import BytesIO
import requests
from torchvision import transforms as T

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def get_prediction(img,threshold=0.5):
  transform = T.Compose([T.ToTensor()])
  img = transform(img)
  pred = model([img])
  pred_classes = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())]
  pred_boxes = [[(i[0],i[1]),(i[2],i[3])]for i in list(pred[0]['boxes'].detach().cpu().numpy())]
  pred_score = list(pred[0]['scores'].detach().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x> threshold][-1]
  pred_box = pred_boxes[:pred_t+1]
  pred_class = pred_classes[:pred_t+1]

  return pred_box, pred_class

In [ ]:
from urllib.request import urlopen

def url_to_image(url,readFlag=cv2.IMREAD_COLOR):
  resp = urlopen(url)
   = np.asarray(bytearray(resp.read()),dtype='uint8')
  image = cv2.imdecode(mg,readFlag)
  return image

IndentationError: ignored

In [ ]:
import random

def object_detection(img_path,threshold = 0.5, rect_th = 3,text_size = 3,text_th =3, url = False):
  if url:
    img = url_to_image(img_path)
  else:
    img = cv2.imread(img_path)

  boxes , pred_class = get_prediction(img,threshold = threshold)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  for i in range(len(boxes)):
    r, g, b = (random.randint(0,255),random.randint(0,255),random.randint(0,255))
    cv2.rectangle(img,boxes[i][0],boxes[i][1],color = (r,g,b),thickness = rect_th)
    cv2.putText(img,pred_class[i],boxes[i][0],cv2.FONT_HERSHEY_SIMPLEX,text_size,(r,g,b),thickness=text_th)
  plt.figure(figsize=(15,15))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()


In [ ]:
!wget https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/10best-cars-group-cropped-1542126037.jpg -O car.jpg

--2022-05-23 06:47:10--  https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/10best-cars-group-cropped-1542126037.jpg
Resolving hips.hearstapps.com (hips.hearstapps.com)... 151.101.0.155, 151.101.64.155, 151.101.128.155, ...
Connecting to hips.hearstapps.com (hips.hearstapps.com)|151.101.0.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2178423 (2.1M) [image/jpeg]
Saving to: ‘car.jpg’

car.jpg             100%[===================>]   2.08M  --.-KB/s    in 0.06s   

2022-05-23 06:47:10 (32.4 MB/s) - ‘car.jpg’ saved [2178423/2178423]



In [ ]:
object_detection('./car.jpg',rect_th=5,text_th=4,text_size=5)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
pip install pytube

In [ ]:
from pytube import YouTube
import os

def downloadYouTube(videourl, path):

    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

In [ ]:
downloadYouTube('https://www.youtube.com/watch?v=OzBYuurGv_c', './testing')

In [ ]:
import os
def in_video_detection(vid_path,threshold = 0.5, rect_th = 3,text_size = 3,text_th =3, url = False):
  cap = cv2.VideoCapture(vid_path)

  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  fps = cap.get(cv2.CAP_PROP_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  now_frame = 0
  total_fps = 30*116
  video_writer = cv2.VideoWriter(os.path.join('data','output.mp4'),fourcc,fps,(width,height))
  while cap.isOpened():
    ret, frame = cap.read()

    boxes, pred_clas = get_prediction(frame,threshold)
    for i in range(len(boxes)):
      r, g, b = (random.randint(0,255),random.randint(0,255),random.randint(0,255))
      cv2.rectangle(frame,boxes[i][0],boxes[i][1],color = (r,g,b),thickness = rect_th)
      cv2.putText(frame,pred_clas[i],boxes[i][0],cv2.FONT_HERSHEY_SIMPLEX,text_size,(r,g,b),thickness=text_th)

    video_writer.write(frame)
    level = (now_frame/total_fps)*100
    now_frame+=1
    print(f" {level} %")
  cap.release()
  video_writer.release()


In [ ]:
in_video_detection('./testing/vid1.mp4',rect_th=5,text_th=4,text_size=5)

 0.0 %
 0.028735632183908046 %
 0.05747126436781609 %
 0.08620689655172414 %
 0.11494252873563218 %
 0.14367816091954022 %
 0.1724137931034483 %
 0.20114942528735633 %
 0.22988505747126436 %
 0.25862068965517243 %
 0.28735632183908044 %
 0.3160919540229885 %
 0.3448275862068966 %
 0.3735632183908046 %
 0.40229885057471265 %
 0.43103448275862066 %
 0.45977011494252873 %
 0.4885057471264368 %
 0.5172413793103449 %
 0.5459770114942528 %
 0.5747126436781609 %
 0.603448275862069 %
 0.632183908045977 %
 0.6609195402298851 %
 0.6896551724137931 %
 0.7183908045977011 %
 0.7471264367816092 %
 0.7758620689655172 %
 0.8045977011494253 %
 0.8333333333333334 %
 0.8620689655172413 %
 0.8908045977011494 %
 0.9195402298850575 %
 0.9482758620689655 %
 0.9770114942528736 %
 1.0057471264367817 %
 1.0344827586206897 %
 1.0632183908045978 %
 1.0919540229885056 %
 1.1206896551724137 %
 1.1494252873563218 %
 1.1781609195402298 %
 1.206896551724138 %
 1.235632183908046 %
 1.264367816091954 %
 1.29310344827586

KeyboardInterrupt: ignored